In [11]:
# ADMM adaptive lasso using ProximalOperators with line and golden section search
# Automatic tuning of learning rate and regularization parameter
# This code only only changes the ratio between test data and train data, and changes the input file
# The original source code from https://github.com/patwa67/AUTALASSO

using ProximalOperators
using DelimitedFiles
using Statistics
using LinearAlgebra

In [12]:
# Function for Golden section search to optimize lambda
function gss_opt(alam, blam, tolgss, Xtesthot, ytest,abscovinv,maxnorm)
  lama =alam*maxnorm # Lower lambda
  lamb =blam*maxnorm # Higher lambda
  gr = (sqrt(5.0) + 1.0) / 2.0 # Golden section ratio
  toladmm = 1e-4 # Convergence tolerance for ADMM
  fc = lasso_admm(Xtrainhot, ytrain, lama, zero(Xtrainhot[1,:]),zero(Xtrainhot[1,:]),f, abscovinv,toladmm)
  lossc= 0.5*norm(Xtesthot*fc[1].-ytest)^2 # Test error for initial lower lambda
  fd = lasso_admm(Xtrainhot, ytrain, lamb, zero(Xtrainhot[1,:]),zero(Xtrainhot[1,:]),f, abscovinv,toladmm)
  lossd= 0.5*norm(Xtesthot*fd[1].-ytest)^2 # Test error for initial higher lambda
  iter = 2
  meanlam = zero(1.0:100.0)
  #meanlam[iter] = (lama+lamb)/2
  meanloss = zero(1.0:100.0)
  #meanloss[1] = max(lossc,lossd)
  #meanloss[iter] = (lossc+lossd)/2
  lamc = lamb - (lamb - lama) / gr
  lamd = lama + (lamb - lama) / gr
  println("lossc =$lossc")
  println("lossd =$lossd")
  println("lambdaa =$lama")
  println("lambdab =$lamb")
  println("lambdac =$lamc")
  println("lambdad =$lamd")
  iter = 2
  nodrun = 0
  while abs(lamc - lamd)/((lamc + lamd)/2.0) > tolgss # Run GSS until convergence
    iter = iter+1
    if iter == 3
    fc = lasso_admm(Xtrainhot, ytrain, lamc, fc[1],fc[2],f, abscovinv,toladmm)
    lossc= 0.5*norm(Xtesthot*fc[1].-ytest)^2 # Test error for initial lower lambda
    fd = lasso_admm(Xtrainhot, ytrain, lamd, fd[1],fd[2],f, abscovinv,toladmm)
    lossd= 0.5*norm(Xtesthot*fd[1].-ytest)^2 # Test error for initial higher lambda
    else
    if nodrun==1
    fc = lasso_admm(Xtrainhot, ytrain, lamc, fc[1],fc[2],f, abscovinv,toladmm)
    lossc= 0.5*norm(Xtesthot*fc[1].-ytest)^2 # Test error for initial lower lambda
    else
    fd = lasso_admm(Xtrainhot, ytrain, lamd, fd[1],fd[2],f, abscovinv,toladmm)
    lossd= 0.5*norm(Xtesthot*fd[1].-ytest)^2 # Test error for initial higher lambda
    end
    end
    meanlam[iter] = (lamc+lamd)/2.0
    meanloss[iter] = (lossc+lossd)/2.0
    # Stop GSS if test MSE is increased two consecutive iterations
    if (meanloss[iter] > meanloss[iter-1])&&(meanloss[iter-1] > meanloss[iter-2])
      break
    end
    if lossc < lossd
      lamb = lamd
      fd=fc
      lossd=lossc
      nodrun=1
    else
      lama = lamc
      fc=fd
      lossc=lossd
      nodrun=0
    end
    lamc = lamb - (lamb - lama) / gr
    lamd = lama + (lamb - lama) / gr
    #println("lossc =$lossc")
    #println("lossd =$lossd")
    println("lambdac =$lamc")
    println("lambdad =$lamd")
  end
  # Final ADMM for optimized lambda
  lamopt = meanlam[iter-2]
  fmean1 = (fc[1]+fd[1])/2.0
  fmean2 = (fc[2]+fd[2])/2.0
  fopt = lasso_admm(Xtrainhot, ytrain, lamopt, fmean1,fmean2,f, abscovinv,toladmm)
  lossopt= 0.5*norm(Xtesthot*fopt[1].-ytest)^2
  acc = cor(Xtesthot*fopt[1],ytest)

  return lamopt,fopt,lossopt,acc
end

gss_opt (generic function with 1 method)

In [13]:
# Function for Proximal ADMM lasso with line search
function lasso_admm(Xtrainhot, ytrain, lam, theta, beta, f, abscovinv,tol; maxit=5000)
  u = zero(theta)
  grad = zero(theta)
  lamw = lam*abscovinv # Regularization parameter times weights
  g = NormL1(lamw) # Regularization function
  c = 0.5
  lr = 1.0
  loss(theta) = 0.5*norm(Xtrainhot*theta-ytrain)^2 # Loss function for line search
  for it = 1:maxit
    # Line search
    it % 8 == 1 && (grad = Xtrainhot'*(Xtrainhot*beta-ytrain))
    while  it % 20 == 2 && loss(theta) > (loss(beta) + grad'*(-beta) + (1.0/(2.0*lr))*norm(-beta)^2)
      lr = lr * c
      #println(lr)
    end
    gam = lr
    # ADMM perform f-update step
    prox!(beta, f, theta - u, gam)
    # ADMM perform g-update step
    prox!(theta, g, beta + u, gam)
    # Stopping criterion for ADMMM
    if norm(beta-theta, Inf) <= tol*(1+norm(u, Inf))
      break
    end
    # Dual update
    u .+= beta - theta
    #print(it)
  end
  return theta,beta,tol
end

lasso_admm (generic function with 1 method)

In [14]:
data_file_path = "/Users/dong/GWAS/AtPolyDB/Silique_22.csv"

"/Users/dong/GWAS/AtPolyDB/Silique_22.csv"

In [15]:
# Read Silique_22.csv, and partition into train and test parts
X = readdlm(data_file_path,',')
ytot = (X[:,1].-mean(X[:,1])) # Center y to mean zero
ytrain = ytot[1:76]
Xtest= X[77:size(X)[1],2:size(X)[2]]
ytest = ytot[77:size(X)[1]]
Xtrain = X[1:76,2:size(X)[2]]

76×214051 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0  …  0.0  0.0  0.0  0.0  0.0  2.0  0.0
 0.0  2.0  0.0  0.0  0.0  2.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  2.0  0.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0     2.0  2.0  0.0  0.0  2.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0     2.0  2.0  0.0  0.0  2.0  0.0  0.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0  …  2.0  2.0  0.0  0.0  2.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  2.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0     0.0  2.0  2.0  0.0  2.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  2.0
 0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0

In [16]:
# One hot encoding training data
Xtrain0 = copy(Xtrain)
Xtrain1 = copy(Xtrain)
Xtrain2 = copy(Xtrain)
Xtrain0[Xtrain0.==1] .= 2
Xtrain0[Xtrain0.==0] .= 1
Xtrain0[Xtrain0.==2] .= 0
Xtrain1[Xtrain1.==2] .= 0
Xtrain2[Xtrain2.==1] .= 0
Xtrain2[Xtrain2.==2] .= 1
Xtrainhot = hcat(Xtrain0,Xtrain1,Xtrain2)
# Set unimportant allocations to zero
Xtrain0 = 0
Xtrain1 = 0
Xtrain2 = 0
Xtrain = 0

0

In [17]:
# One hot encoding test data
Xtest0 = copy(Xtest)
Xtest1 = copy(Xtest)
Xtest2 = copy(Xtest)
Xtest0[Xtest0.==1] .= 2
Xtest0[Xtest0.==0] .= 1
Xtest0[Xtest0.==2] .= 0
Xtest1[Xtest1.==2] .= 0
Xtest2[Xtest2.==1] .= 0
Xtest2[Xtest2.==2] .= 1
Xtesthot = hcat(Xtest0,Xtest1,Xtest2)
# Set unimportant allocations to zero
Xtest0 = 0
Xtest1 = 0
Xtest2 = 0
Xtest = 0

0

In [18]:
# Factor for initial lower lambda
alam = 0.0001
# Factor for initial upper lambda
blam = 1.0
# Convergence factor for lambda in gss_opt
tolgss = 0.01
# Find lambda where all reg coeff are zero
maxnorm = norm(Xtrainhot'*ytrain, Inf)
# The least squares loss function
f = LeastSquares(Xtrainhot, ytrain)
# Inverse covariances to be used as weights in the adaptive lasso
abscovinv = 1.0./abs.(cov(Xtrainhot,ytrain))

642153×1 Matrix{Float64}:
  145.40816326530614
  363.0573248407638
  145.40816326530614
  147.6683937823835
 2850.000000000015
 1075.4716981132058
 2714.2857142857147
  123.91304347826082
   56.603773584905646
  150.39577836411615
  306.45161290322585
   56.603773584905646
  306.45161290322585
    ⋮
   87.15596330275226
  303.1914893617024
  155.31335149863753
  123.37662337662337
 4071.428571428539
  105.94795539033458
   65.66820276497694
  123.37662337662337
 1036.363636363638
   44.91725768321513
  255.60538116591945
   87.55760368663596

In [19]:
# Run AUTALASSO with timing
@time res = gss_opt(alam, blam, tolgss, Xtesthot, ytest,abscovinv,maxnorm)


lossc =0.29980000000000023
lossd =0.29980000000000023
lambdaa =0.00040399999999999925
lambdab =4.039999999999992
lambdac =1.5433923711818767
lambdad =2.4970116288181154
lambdac =2.4970116288181154
lambdad =3.0863807423637537
lambdac =3.0863807423637537
lambdad =3.450630886454354
lambdac =3.450630886454354
lambdad =3.675749855909392
lambdac =3.6757498559093915
lambdad =3.8148810305449543
lambdac =3.8148810305449543
lambdad =3.9008688253644292
lambdac =3.9008688253644292
lambdad =3.954012205180517
lambdac =3.954012205180517
lambdad =3.986856620183904
  3.323985 seconds (67.26 k allocations: 495.188 MiB, 6.88% gc time, 5.22% compilation time)


(3.745315443227173, ([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-6.045965714518431e-6, -5.641138173029248e-6, -6.045965714518431e-6, 7.406858911540581e-6, -1.1083301087344566e-5, 9.569997130642882e-6, 5.858838606487282e-7, -6.882034233157697e-6, 3.708453269135603e-6, 8.90704241673923e-6  …  1.2544743364761857e-5, 1.6908124448322681e-6, -1.156182402331507e-6, -2.5091410004174364e-6, 1.0118541644565937e-6, 1.6908124448322681e-6, -2.6141015459363315e-6, -5.0649886049747295e-6, 4.585634206244471e-6, 5.145112734306068e-6], 0.0001), 0.29980000000000023, NaN)

In [20]:
output_dir = "/Users/dong/GWAS/Autalasso/" 


"/Users/dong/GWAS/Autalasso/"

In [21]:
# Save regression coefficients, lambda, MSE and ACC to text files
writedlm(joinpath(output_dir, "outbeta_Silique22.txt"), res[2][1])
writedlm(joinpath(output_dir, "outlambda_Silique22.txt"), res[1])
writedlm(joinpath(output_dir, "outMSE_Silique22.txt"), res[3]/length(ytest)*2)
writedlm(joinpath(output_dir, "outACC_Silique22.txt"), res[4])

In [22]:
# Predict observations for test data
ytesthat = Xtesthot*res[2][1]

19-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0